In [4]:
from nltk.tokenize import word_tokenize
import nltk
import scipy as sp

AttributeError: module 'nltk' has no attribute 'data'

1. term frequency
 - tf
 - 어떤 단어가 문서 내에서 자주 등장할수록, 중요도가 높아진다 

2. inverse document frequency
 - idf
 - 비교하는 모든 문서 내에서 만약에 같은 단어가 존재한다면, 핵심 어휘일수 있다
 - 그러나, 문서간의 비교에서는 중요한 단어가 아닐수있다
 
3. 결론
 - 텍스트 마이닝에서 사용하는 기법, 단어별에 가중치를 부과하는 방식

In [64]:
# 함수 구성
'''
t : 단어 , 'a' or 'b' or 'c'
d : 문장을 가진 한세트
D : 문장 리스트를 맴버로 가진 리스트(비교할 문장덩어리)
'''
def tfidf( t, d, D ):
    #print( "float( %s.count(%s) ) / sum( %s.count(w) for w in set(%s) )" % (d,t,d,d) )
    #print( "%s / %s" % (float( d.count(t) ), sum( d.count(w) for w in set(d) )) )
    tf = float( d.count(t) ) / sum( d.count(w) for w in set(d) )
    print( 'sp.log( %s/%s)' %  (float(len(D)), (len([ doc for doc in D if t in doc ]))) )
    idf = sp.log( float(len(D)) / (len([ doc for doc in D if t in doc ]))   )
    return tf, idf

In [65]:
# d.count(w) for w in set(d)
# 문장안에서 단어별 발생된 총 빈도수의 합산
print( [ ['a'].count(w) for w in set(['a']) ] )
print( [ ['a','b','b'].count(w) for w in set(['a','b','b']) ] )
print( [ ['a','b','c'].count(w) for w in set(['a','b','c']) ] )

[1]
[1, 2]
[1, 1, 1]


In [66]:
print( sum( [ ['a'].count(w) for w in set(['a']) ] ))
print( sum( [ ['a','b','b'].count(w) for w in set(['a','b','b']) ] ))
print( sum( [ ['a','b','c'].count(w) for w in set(['a','b','c']) ] ))

1
3
3


In [67]:
# d.count(t)
print( float(['a'].count( 'a' )) )
print( float(['a','b','b'].count( 'b' )) )
print( float(['a','b','c'].count( 'c' )) )
print( float(['a','b','c'].count( 'a' )) )

1.0
2.0
1.0
1.0


In [68]:
# 각각 문장
a, abb, abc = ['a'],['a','b','b'],['a','b','c']
# 여러 문장을 모아 두었다
D = [ a, abb, abc ]

In [69]:
# 문장 전체의 단어 빈도수 대비 해당 단어의 빈도수의 비율 : tf
# 단어 빈도가 1인데 a라는 단어도 1이므로   tf = 1
print( tfidf( 'a', a, D ) )
# 단어 빈도수가 3인데  b 는 2번 등장했으므로 tf=0.67
print( tfidf( 'b', abb, D ) )
# 단어 빈도수가 3인데  b 는 1번 등장했으므로 tf=0.33
print( tfidf( 'a', abc, D ) )
print( tfidf( 'b', abc, D ) )
print( tfidf( 'c', abc, D ) )

sp.log( 3.0/3)
(1.0, 0.0)
sp.log( 3.0/2)
(0.6666666666666666, 0.4054651081081644)
sp.log( 3.0/3)
(0.3333333333333333, 0.0)
sp.log( 3.0/2)
(0.3333333333333333, 0.4054651081081644)
sp.log( 3.0/1)
(0.3333333333333333, 1.0986122886681098)


In [70]:
# 라이브러리로 처리
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer( min_df=1, decode_error='ignore' )

In [71]:
# 형태소 분석기
from konlpy.tag import Okt
t = Okt()

In [72]:
contents = ['성건이랑 술마시고 싶지만 바쁜데 어떻하죠?',
            '성건이는 공원에서 산책하고 노는 것을 싫어해요',
            '성건이는 공원에서 노는 것도 싫어해요. 이상해요.',
            '먼 곳으로 여행을 떠나고 싶은데 너무 바빠서 그러질 못하고 있어요.'
           ]

In [73]:
contents_tokens = [ t.morphs( doc ) for doc in contents ]
contents_tokens

[['성', '건', '이랑', '술', '마시고', '싶지만', '바쁜데', '어떻하죠', '?'],
 ['성', '건', '이', '는', '공원', '에서', '산책', '하고', '노', '는', '것', '을', '싫어해요'],
 ['성',
  '건',
  '이',
  '는',
  '공원',
  '에서',
  '노',
  '는',
  '것',
  '도',
  '싫어해요',
  '.',
  '이상해요',
  '.'],
 ['먼',
  '곳',
  '으로',
  '여행',
  '을',
  '떠나고',
  '싶은데',
  '너무',
  '바빠서',
  '그러질',
  '못',
  '하고',
  '있어요',
  '.']]

In [74]:
len(contents_tokens), len(contents_tokens[0])

(4, 9)

In [75]:
# 형태소 기반으로 문장을 재구성
# => "품질 이랑 술 마시고 싶지만 바쁜데 어떻하죠 ?"
# contents_for_vectorize 여기에 재구성한 문장을 담는다
contents_for_vectorize = [ ' '.join(content) for content in contents_tokens ]

# for content in contents_tokens:    
#     tmp = ''
#     print( ' '.join(content) )
#     for word in content:
#         tmp = tmp + ' ' + word
        
#     contents_for_vectorize.append( tmp )

In [76]:
contents_for_vectorize

['성 건 이랑 술 마시고 싶지만 바쁜데 어떻하죠 ?',
 '성 건 이 는 공원 에서 산책 하고 노 는 것 을 싫어해요',
 '성 건 이 는 공원 에서 노 는 것 도 싫어해요 . 이상해요 .',
 '먼 곳 으로 여행 을 떠나고 싶은데 너무 바빠서 그러질 못 하고 있어요 .']

In [77]:
# 백터화 변환
X = vectorizer.fit_transform( contents_for_vectorize )
X.shape

(4, 19)

In [78]:
# 데이터의 크기를 변수로 받는다 
num_samples, num_features = X.shape
num_samples, num_features

(4, 19)

In [79]:
# 백터의 특성화값 -> 컬럼-> 변수, 특성
vectorizer.get_feature_names()[:3]

['공원', '그러질', '너무']

In [91]:
# 신규 문장 -> 백터화
# 1. 신문장
#new_post = ['성건이랑 공원에서 산책하고 놀고 싶어요']
new_post = ['근처 공원에서 성건이랑 놀러가고 싶어요']
# 2. 형태소 분리
new_post_tokens = [ t.morphs( doc ) for doc in new_post ]
new_post_tokens

[['근처', '공원', '에서', '성', '건', '이랑', '놀러', '가고', '싶어요']]

In [92]:
# 3. 형태소 단위로 문장 재구성
new_post_for_vectorize = [ ' '.join(content) for content in new_post_tokens ]
new_post_for_vectorize

['근처 공원 에서 성 건 이랑 놀러 가고 싶어요']

In [93]:
# 4. 백터화
new_post_vec = vectorizer.transform( new_post_for_vectorize )
new_post_vec.shape

(1, 19)

In [94]:
# 거리 계산(백터간)=> 정규화를 적용하여
def dist_norm( v1, v2 ):
    # 각각 백터 문장의 정규화
    v1_norm = v1 / sp.linalg.norm( v1.toarray() )
    v2_norm = v2 / sp.linalg.norm( v2.toarray() )
    # 두 문장의 거리 (차이) 계산
    delta = v1_norm - v2_norm
    # 결과 리턴
    return sp.linalg.norm( delta.toarray() )

In [95]:
# 유사도 검사
best_doc      = None
best_distance = 1000
best_index    = None
for i in range( num_samples ):    
    # 비교 문장의 백터값 획득
    post_vec = X.getrow(i)   
    # 거리 계산
    d        = dist_norm( post_vec,  new_post_vec )
    print( d )
    # 기준값보다 작을때 처리
    if d < best_distance:
        # 거리값
        best_distance = d
        # 유사한 인덱스
        best_index    = i
        # 유사한 문장
        best_doc      = contents[ best_index ]

1.1844028684759158
1.0539963851720047
1.009546132854007
1.4142135623730951


In [96]:
# 결과 확인
best_distance

1.009546132854007

In [97]:
best_index

2

In [98]:
best_doc

'성건이는 공원에서 노는 것도 싫어해요. 이상해요.'

In [99]:
contents

['성건이랑 술마시고 싶지만 바쁜데 어떻하죠?',
 '성건이는 공원에서 산책하고 노는 것을 싫어해요',
 '성건이는 공원에서 노는 것도 싫어해요. 이상해요.',
 '먼 곳으로 여행을 떠나고 싶은데 너무 바빠서 그러질 못하고 있어요.']

In [100]:
new_post

['근처 공원에서 성건이랑 놀러가고 싶어요']